In [1]:
#importing the required modules.
import random
import json
import pickle
import numpy as np
import nltk
  
from keras.models import Sequential
from nltk.stem import WordNetLemmatizer
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
  
  
lemmatizer = WordNetLemmatizer()
  
# reading the json.intense file
intents = json.loads(open("intents.json").read())
  
# creating empty lists to store data
words = []
classes = []
documents = []
ignore_letters = ["?", "!", ".", ","]
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # separating words from patterns
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)  # and adding them to words list
          
        # associating patterns with respective tags
        documents.append(((word_list), intent['tag']))
  
        # appending the tags to the class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
  
# storing the root words or lemma
words = [lemmatizer.lemmatize(word)
         for word in words if word not in ignore_letters]
words = sorted(set(words))
  
# saving the words and classes list to binary files
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

2022-12-12 12:10:45.060914: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-12 12:10:45.400035: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-12 12:10:45.400068: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-12 12:10:45.440909: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-12 12:10:46.342533: W tensorflow/stream_executor/platform/de

In [2]:
# we need numerical values of the
# words because a neural network
# needs numerical values to work with
training = []
output_empty = [0]*len(classes)
for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(
        word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
          
    # making a copy of the output_empty
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])
random.shuffle(training)
training = np.array(training)
  
# splitting the data
train_x = list(training[:, 0])
train_y = list(training[:, 1])

/tmp/ipykernel_4208/330460699.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [3]:
# creating a Sequential machine learning model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]), ),
                activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), 
                activation='softmax'))
  
# compiling the model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd, metrics=['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y),
                 epochs=200, batch_size=5, verbose=1)
  
# saving the model
model.save("chatbotmodel.h5", hist)
  
# print statement to show the 
# successful training of the Chatbot model
print("Yay!")

Epoch 1/200


2022-12-07 12:51:24.331774: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-07 12:51:24.331872: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-07 12:51:24.331907: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-12-07 12:51:24.331937: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-12-07 12:51:24.331965: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

31/31 [==============================] - 0s 870us/step - loss: 3.9540 - accuracy: 0.0065
Epoch 2/200
31/31 [==============================] - 0s 906us/step - loss: 3.9080 - accuracy: 0.0261
Epoch 3/200
31/31 [==============================] - 0s 821us/step - loss: 3.8336 - accuracy: 0.0392
Epoch 4/200
31/31 [==============================] - 0s 862us/step - loss: 3.7051 - accuracy: 0.1242
Epoch 5/200
31/31 [==============================] - 0s 822us/step - loss: 3.6451 - accuracy: 0.1111
Epoch 6/200
31/31 [==============================] - 0s 853us/step - loss: 3.5190 - accuracy: 0.1176
Epoch 7/200
31/31 [==============================] - 0s 884us/step - loss: 3.3364 - accuracy: 0.1634
Epoch 8/200
31/31 [==============================] - 0s 829us/step - loss: 3.3073 - accuracy: 0.1438
Epoch 9/200
31/31 [==============================] - 0s 868us/step - loss: 3.1234 - accuracy: 0.1765
Epoch 10/200
31/31 [==============================] - 0s 840us/step - loss: 2.8924 - accuracy: 0.2745
E

31/31 [==============================] - 0s 1ms/step - loss: 0.3560 - accuracy: 0.8824
Epoch 162/200
31/31 [==============================] - 0s 821us/step - loss: 0.2273 - accuracy: 0.9085
Epoch 163/200
31/31 [==============================] - 0s 809us/step - loss: 0.3190 - accuracy: 0.9085
Epoch 164/200
31/31 [==============================] - 0s 829us/step - loss: 0.3013 - accuracy: 0.8954
Epoch 165/200
31/31 [==============================] - 0s 845us/step - loss: 0.3118 - accuracy: 0.9085
Epoch 166/200
31/31 [==============================] - 0s 958us/step - loss: 0.2805 - accuracy: 0.8824
Epoch 167/200
31/31 [==============================] - 0s 965us/step - loss: 0.2116 - accuracy: 0.9346
Epoch 168/200
31/31 [==============================] - 0s 966us/step - loss: 0.3636 - accuracy: 0.8758
Epoch 169/200
31/31 [==============================] - 0s 954us/step - loss: 0.2352 - accuracy: 0.9085
Epoch 170/200
31/31 [==============================] - 0s 904us/step - loss: 0.3411 - acc